In [2]:
# 1. Setup and initialization
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.query import MetadataQuery

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

# Initialize clients
wcd_url = os.environ["WCD_URL"]
wcd_api_key = os.environ["WCD_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,                                    
    auth_credentials=Auth.api_key(wcd_api_key),
)
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [10]:
# 2. Define the query
query = "what is the capital of Germany?"

# 3. Get embedding for the query
response = openai_client.embeddings.create(
    model="text-embedding-3-large",
    input=query
)
query_embedding = response.data[0].embedding

In [11]:
prompt = f"Your job is to decide if this use query would be better answered with extra context from Tesla's manual and if so output 1 otherwise output 0."

response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": query}
    ],
    temperature=0
)

answer = response.choices[0].message.content

print(answer)

0


In [12]:
context_str = ""

if answer == "1":
    # 4. Search for similar texts in Weaviate
    collection_name = "Vladimir"

    collection = client.collections.get(collection_name)
    similar_texts = collection.query.near_vector(
        near_vector=query_embedding,
        limit=3,
        return_properties=["text"],
        return_metadata=MetadataQuery(distance=True)
    )

    print("\nRetrieved contexts:")
    for i, doc in enumerate(similar_texts.objects, 1):
        print(f"\n{i}. Distance: {doc.metadata.distance}")
        print(f"Text: {doc.properties['text']}")
        print("\n\n")

    context_str = "\n\n---\n\n".join([doc.properties["text"] for doc in similar_texts.objects])

In [14]:
# 5. Prepare context for GPT

prompt = query
if answer == "1":
    prompt = f"""Answer the question using ONLY the information provided in the context below. 
    Do not add any general knowledge or information not contained in the context."

    Context:
    {context_str}

    Question: {query}

    Answer:"""

# 6. Generate answer using GPT-4
response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided context."},
        {"role": "user", "content": prompt}
    ],
    temperature=0
)

answer = response.choices[0].message.content

# 7. Print results
print(f"Prompt: {prompt} \n\n----------------------------------\n\n")
print(f"Question: {query}\n\nAnswer: {answer}")

Prompt: what is the capital of Germany? 

----------------------------------


Question: what is the capital of Germany?

Answer: The capital of Germany is Berlin.
